PEMROSESAN TEKS SCRAPPING GOOGLE SCHOLAR
Fadhilah Nuria Shinta 22031554003
Analicia 22031554007
Riva Dian A. 22031554043

In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=2bf79ed7bb8ffffdfc90f46bf3d8dc96cb7ba31ee45f7abc163881a46686388e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
# Jalankan code ini dijupyter

from langdetect import detect
import requests
from bs4 import BeautifulSoup
import csv

def web_scrape(query, start_year=None, end_year=None, pdf_only=False):
    base_url = "https://scholar.google.com/scholar"
    params = {
        "q": query
    }

    if start_year and end_year:
        params["as_ylo"] = start_year
        params["as_yhi"] = end_year

    if pdf_only:
        params["as_vis"] = "1"  # "1" mengindikasikan hanya mencari artikel PDF yang open source

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }

    response = requests.get(base_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    results = soup.find_all('div', class_='gs_ri')

    for idx, result in enumerate(results, start=1):
        title_tag = result.find('h3', class_='gs_rt')
        if title_tag:
            title = title_tag.text
            authors = result.find('div', class_='gs_a').text.split('-')[0].strip()
            source = result.find('div', class_='gs_a').text.split('-')[1].strip()

            abstract = result.find('div', class_='gs_rs')
            if abstract:
                paper = abstract.text
            else:
                paper = "N/A"

            year = result.find('div', class_='gs_a').text.split('-')[-1].strip()
            url_article = title_tag.a['href']

            # Mendeteksi bahasa dalam paper
            language = detect(paper)  # Use the detect function to determine the language

            articles.append([title, authors, year, source, url_article, language, paper])

    return articles  # Return the results from the function

# Define the save_to_csv function to save results to a CSV file
def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Write header
        csvwriter.writerow(["Title", "Authors", "Source", "Year", "URL", "Language", "Abstract"])
        # Write data
        csvwriter.writerows(data)

if __name__ == "__main__":
    # Input kata kunci dari pengguna
    search_query = input("Masukkan kata kunci: ")

    # Input tahun awal (start year) dan tahun akhir (end year) dari pengguna
    start_year = input("Masukkan tahun awal (opsional): ")
    end_year = input("Masukkan tahun akhir (opsional): ")

    # Input apakah hanya mencari artikel PDF yang open source
    pdf_only_input = input("Cari hanya artikel PDF yang open source? (y/n): ")
    pdf_only = pdf_only_input.lower() == "y"

    # Konversi input tahun menjadi integer jika tersedia
    if start_year:
        start_year = int(start_year)
    if end_year:
        end_year = int(end_year)

    # Lakukan web scraping dengan kriteria tahun dan jenis artikel
    search_results = web_scrape(search_query, start_year, end_year, pdf_only)

    # Cek apakah ada hasil pencarian
    if search_results:
        # Simpan data dalam file CSV
        output_filename = f"{search_query.replace(' ', '_')}_results.csv"
        save_to_csv(search_results, output_filename)
        print(f"Hasil pencarian telah disimpan dalam file {output_filename}")
    else:
        print("Tidak ada hasil pencarian yang ditemukan.")


Masukkan kata kunci: speech to text
Masukkan tahun awal (opsional): 2023
Masukkan tahun akhir (opsional): 2023
Cari hanya artikel PDF yang open source? (y/n): y
Tidak ada hasil pencarian yang ditemukan.


In [4]:
import pandas as pd
file_path = 'unesa_results.csv'
df=pd.read_csv(file_path)
df

FileNotFoundError: ignored

https://colab.research.google.com/drive/1GN5RMmeIOtt_NhwQp0Sj40jz5xbSsDWy?usp=sharing

In [5]:
# Jalankan code ini di google collab
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrapweb(katakunci):
    hasil = []
    url = 'https://google.com/search?q=' + katakunci
    request_result = requests.get(url)
    soup = BeautifulSoup(request_result.text, "html.parser")
    print(soup)
    heading_object = soup.find_all('h3')
    for info in heading_object:
        hasil.append(info)
        print(info.getText())
        print("------")
    return hasil

def scrapJournal(theKeyword, start_year=None, end_year=None, languages=None):
    keyword = theKeyword.replace(' ', '%20')
    page_number = 1
    results = []

    while True:
        URL = f"https://scholar.google.com/scholar?q={keyword}&start={page_number * 10}"
        page = requests.get(URL)

        if page.status_code == 200:
            soup = BeautifulSoup(page.text, "html.parser")
            journal_items = soup.find_all('div', class_='gs_ri')

            if not journal_items:
                break

            for item in journal_items:
                title = item.find('h3', class_='gs_rt')
                author = item.find('div', class_='gs_a')
                link = item.find('div', class_='gs_ggs gs_fl')

                if title:
                    title = title.text.strip()

                if author:
                    author = author.text.strip()

                if link:
                    link = link.a['href']

                # Tambahkan data ke hasil
                results.append({'Title': title, 'Author': author, 'DocumentLink': link})

            page_number += 1
        else:
            print(f"Gagal mengakses halaman {page_number}. Kode status: {page.status_code}")

    if results:
        df = pd.DataFrame(results)
        return df
    else:
        print("Tidak ada hasil yang ditemukan.")
        return None

# Input dari pengguna
keywords = input("Masukkan kata kunci: ")
start_year = input("Masukkan tahun awal (kosongkan jika tidak ada): ")
end_year = input("Masukkan tahun akhir (kosongkan jika tidak ada): ")
languages = input("Masukkan bahasa (pisahkan dengan koma jika lebih dari satu): ").split(',')

# Contoh penggunaan scrapJournal dengan input pengguna
hasil_scraping = scrapJournal(keywords, start_year, end_year, languages)
if hasil_scraping is not None:
    print(hasil_scraping)

    # Menyimpan DataFrame ke dalam file CSV
    hasil_scraping.to_csv("hasil_scraping.csv", index=False)
    print("Data telah disimpan dalam file CSV: hasil_scraping.csv")
else:
    print("Tidak ada hasil yang ditemukan.")

Masukkan kata kunci: speech to text
Masukkan tahun awal (kosongkan jika tidak ada): 2023
Masukkan tahun akhir (kosongkan jika tidak ada): 2023
Masukkan bahasa (pisahkan dengan koma jika lebih dari satu): Bahasa Indonesia 
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kode status: 429
Gagal mengakses halaman 1. Kod

KeyboardInterrupt: ignored